In [1]:
import pandas as pd
import requests
import json

my_free_api_key = '9a2c24d00b614ff1d74a08f03054ef53'

In [ ]:
def weather_forcast(list_of_cities):
      
    cities_list  = []
    weather_list = []
    
    for city in list_of_cities :
        city_r = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={my_free_api_key}&units=metric")
        city_j = city_r.json()
        
        cities_list.append(city_j['city'])
        

        for detail in city_j['list']:
            weather_details_dic = {}
            weather_details_dic["city_id"] = city_j['city']['id']
            weather_details_dic["city_name"] = city_j['city']['name']
            weather_details_dic["temp"] = detail['main']['temp']
            weather_details_dic["temp_min"] = detail['main']['temp_min']
            weather_details_dic["temp_max"] = detail['main']['temp_max']
            weather_details_dic["feels_like"] = detail['main']['feels_like']
            weather_details_dic["pressure"] = detail['main']['pressure']
            weather_details_dic["humidity"] = detail['main']['humidity']
            weather_details_dic["weather"] = detail['weather'][0]['main']
            weather_details_dic["weather_description"] = detail['weather'][0]['description']
            weather_details_dic["clouds"] = detail['clouds']['all']
            weather_details_dic["wind_speed"] = detail['wind']['speed']
            weather_details_dic["date_time"] = detail['dt_txt']
            weather_list.append(weather_details_dic)
    df_weather = pd.DataFrame(weather_list)
    df_cities = pd.json_normalize(cities_list)
    
    return df_weather,df_cities

In [2]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/term"

querystring = {"q":"london","limit":"10"}

headers = {
	"X-RapidAPI-Key": "5271551a49mshc194f542b6dc600p113722jsn431b32cb69ab",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"items":[{"icao":"EGSS","iata":"STN","name":"London, London Stansted","shortName":"Stansted","municipalityName":"London","location":{"lat":51.885,"lon":0.234999},"countryCode":"GB"},{"icao":"EGLL","iata":"LHR","name":"London, London Heathrow","shortName":"Heathrow","municipalityName":"London","location":{"lat":51.4706,"lon":-0.461941},"countryCode":"GB"},{"icao":"EGLC","iata":"LCY","name":"London, London City","shortName":"City","municipalityName":"London","location":{"lat":51.5053,"lon":0.055277},"countryCode":"GB"},{"icao":"EGKK","iata":"LGW","name":"London, London Gatwick","shortName":"Gatwick","municipalityName":"London","location":{"lat":51.1481,"lon":-0.190277},"countryCode":"GB"},{"icao":"EGGW","iata":"LTN","name":"London, London Luton","shortName":"Luton","municipalityName":"London","location":{"lat":51.8747,"lon":-0.368333},"countryCode":"GB"},{"icao":"KLOZ","iata":"LOZ","name":"London, London-Corbin /Magee Field","shortName":"-Corbin /Magee Field","municipalityName":"London"

In [14]:
airports_j = response.json()
# airports_j['items']

right_airports = [airport_j for airport_j in airports_j['items'] if airport_j['countryCode']=='GB'] 
right_airports


[{'icao': 'EGSS',
  'iata': 'STN',
  'name': 'London, London Stansted',
  'shortName': 'Stansted',
  'municipalityName': 'London',
  'location': {'lat': 51.885, 'lon': 0.234999},
  'countryCode': 'GB'},
 {'icao': 'EGLL',
  'iata': 'LHR',
  'name': 'London, London Heathrow',
  'shortName': 'Heathrow',
  'municipalityName': 'London',
  'location': {'lat': 51.4706, 'lon': -0.461941},
  'countryCode': 'GB'},
 {'icao': 'EGLC',
  'iata': 'LCY',
  'name': 'London, London City',
  'shortName': 'City',
  'municipalityName': 'London',
  'location': {'lat': 51.5053, 'lon': 0.055277},
  'countryCode': 'GB'},
 {'icao': 'EGKK',
  'iata': 'LGW',
  'name': 'London, London Gatwick',
  'shortName': 'Gatwick',
  'municipalityName': 'London',
  'location': {'lat': 51.1481, 'lon': -0.190277},
  'countryCode': 'GB'},
 {'icao': 'EGGW',
  'iata': 'LTN',
  'name': 'London, London Luton',
  'shortName': 'Luton',
  'municipalityName': 'London',
  'location': {'lat': 51.8747, 'lon': -0.368333},
  'countryCode': '